√ The variances files <br>
<b> Need to run the save_avg notebook first to get the averages, then run this one to get the coeffs <br>
Put the notebooks (save_avg.ipynb, save_var.ipynb, and this one) and the saved files (already saved: *var*.npy) under a new folder, or it will be too messy

In [1]:
import numpy as np

In [4]:
# MAD, Rh=1
maTe_avg1 = np.load("ma_rh1_avgTe.npy", allow_pickle=True).item()['Te']
maTe_var1 = np.load("ma_rh1_varTe.npy", allow_pickle=True)
maR_avg1 = np.load("ma_avgR1.npy", allow_pickle=True).item()['R']

In [5]:
# MAD, Rh=160
maTe_avg160 = np.load("ma_rh160_avgTe.npy", allow_pickle=True).item()['Te']
maTe_var160 = np.load("ma_rh160_varTe.npy", allow_pickle=True)
maR_avg160 = np.load("ma_avgR160.npy", allow_pickle=True).item()['R']

In [6]:
# MAD
maRHO_avg = np.load("ma_avgRHO.npy", allow_pickle=True).item()['RHO']
maRHO_var = np.load("ma_varRHO.npy", allow_pickle=True)

maPg_avg = np.load("ma_avgPg.npy", allow_pickle=True).item()['Pg']
maPg_var = np.load("ma_varPg.npy", allow_pickle=True)

mabsq_avg = np.load("ma_avgbsq.npy", allow_pickle=True).item()['bsq']
mabsq_var = np.load("ma_varbsq.npy", allow_pickle=True)

In [7]:
# SANE, Rh=1
saTe_avg1 = np.load("sa_rh1_avgTe.npy", allow_pickle=True).item()['Te']
saTe_var1 = np.load("sa_rh1_varTe.npy", allow_pickle=True)
saR_avg1 = np.load("sa_avgR1.npy", allow_pickle=True).item()['R']
type(saTe_avg1)

numpy.ndarray

In [8]:
# SANE, Rh=160
saTe_avg160 = np.load("sa_rh160_avgTe.npy", allow_pickle=True).item()['Te']
saTe_var160 = np.load("sa_rh160_varTe.npy", allow_pickle=True)
saR_avg160 = np.load("sa_avgR160.npy", allow_pickle=True).item()['R']

# SANE
saRHO_avg = np.load("sa_avgRHO.npy", allow_pickle=True).item()['RHO']
saRHO_var = np.load("sa_varRHO.npy", allow_pickle=True)

saPg_avg = np.load("sa_avgPg.npy", allow_pickle=True).item()['Pg']
saPg_var = np.load("sa_varPg.npy", allow_pickle=True)

sabsq_avg = np.load("sa_avgbsq.npy", allow_pickle=True).item()['bsq']
sabsq_var = np.load("sa_varbsq.npy", allow_pickle=True)

In [10]:
# cross terms
macov_rhoPg = np.load("ma_avgRHOPg.npy", allow_pickle=True).item()['rhoPg']
macov_PgPmag = np.load("ma_avgPgPmag.npy", allow_pickle=True).item()['PgPmag']
macov_rhoPmag = np.load("ma_avgRHOPmag.npy", allow_pickle=True).item()['rhoPmag']

sacov_rhoPg = np.load("sa_avgRHOPg.npy", allow_pickle=True).item()['rhoPg']
sacov_PgPmag = np.load("sa_avgPgPmag.npy", allow_pickle=True).item()['PgPmag']
sacov_rhoPmag = np.load("sa_avgRHOPmag.npy", allow_pickle=True).item()['rhoPmag']

### Newly derived coefficients<br>
$a = y(\gamma_{i} - 1)$ <br>
$b = z(\gamma_{e} - 1)$ <br>
$c = \frac{m_{p}(\gamma_{i} - 1)(\gamma_{e}-1)}{k_{B}(\gamma - 1)}$ <br>
$C[Pg -> Te] = A = \frac{<Te>}{<Pg>}-\frac{2b<Te><Pg>(R_{h} - <R>)}{(<P_{mag}>^{2}+<Pg>^{2})(a+b<R>)}$ <br>
$C[\rho -> Te] = B = -\frac{<Te>}{<\rho>}$ <br>
$C[B^{2} -> Te] = C = \frac{b<Te><Pg>}{2\mu_{0}(<P_{mag}>^{2}+<Pg>^{2})(a+b<R>)}$

In [11]:
# some constants
cl = 29979245800.0 # speed of light in cgs
mp = 1.67262171e-24
kb = 1.3806505e-16
mu = 1.

In [12]:
# constants used for the functions
y = 1.
z = 3.
gamma = 4/3
gamma_i = 4/3
gamma_e = 7/3
a = y*(gamma_i - 1)
b = z*(gamma_e - 1)
c = mp*(gamma_e - 1)*(gamma_i - 1)/(kb*(gamma - 1))

In [13]:
# R
def R(dump, Rh, Rl=1):
    beta = dump['beta']
    return Rl/(1+beta**2)+Rh*beta**2/(1+beta**2)

In [19]:
def CPg(avgTe, avgPg, avgPmag, avgR, Rh):
    term1 = avgTe/avgPg
    term2 = 2*b*avgTe*avgPg*(Rh-avgR)/((avgPmag**2+avgPg**2)*(a+b*avgR))
    return term1-term2

In [15]:
def Crho(avgTe, avgrho):
    return -avgTe/avgrho

In [16]:
def Cbsq(avgTe, avgPg, avgPmag, avgR):
    nume = b*avgTe*avgPg
    denom = 2*mu*(avgPmag**2+avgPg**2)*(a+b*avgR)
    return nume/denom

In [20]:
# MAD, Rh=1
maPmag_avg = 1/(2*mu)*mabsq_avg
ma_CPg1 = CPg(maTe_avg1, maPg_avg, maPmag_avg, maR_avg1, Rh=1.0)
ma_Crho1 = Crho(maTe_avg1, maRHO_avg)
ma_Cbsq1 = Cbsq(maTe_avg1, maPg_avg, maPmag_avg, maR_avg1)

In [23]:
type(ma_CPg1)

numpy.ndarray

In [24]:
np.save('ma_CPg1.npy', ma_CPg1)
np.save('ma_Crho1.npy', ma_Crho1)
np.save('ma_Cbsq1.npy', ma_Cbsq1)

In [26]:
# MAD, Rh=160
maPmag_avg = 1/(2*mu)*mabsq_avg
ma_CPg160 = CPg(maTe_avg160, maPg_avg, maPmag_avg, maR_avg160, Rh=160.0)
ma_Crho160 = Crho(maTe_avg160, maRHO_avg)
ma_Cbsq160 = Cbsq(maTe_avg160, maPg_avg, maPmag_avg, maR_avg160)

In [27]:
np.save('ma_CPg160.npy', ma_CPg160)
np.save('ma_Crho160.npy', ma_Crho160)
np.save('ma_Cbsq160.npy', ma_Cbsq160)

In [28]:
# SANE, Rh=1
saPmag_avg = 1/(2*mu)*sabsq_avg
sa_CPg1 = CPg(saTe_avg1, saPg_avg, saPmag_avg, saR_avg1, Rh=1.0)
sa_Crho1 = Crho(saTe_avg1, saRHO_avg)
sa_Cbsq1 = Cbsq(saTe_avg1, saPg_avg, saPmag_avg, saR_avg1)

In [29]:
np.save('sa_CPg1.npy', sa_CPg1)
np.save('sa_Crho1.npy', sa_Crho1)
np.save('sa_Cbsq1.npy', sa_Cbsq1)

In [31]:
# saD, Rh=1
saPmag_avg = 1/(2*mu)*sabsq_avg
sa_CPg160 = CPg(saTe_avg160, saPg_avg, saPmag_avg, saR_avg160, Rh=160.0)
sa_Crho160 = Crho(saTe_avg160, saRHO_avg)
sa_Cbsq160 = Cbsq(saTe_avg160, saPg_avg, saPmag_avg, saR_avg160)

In [32]:
np.save('sa_CPg160.npy', sa_CPg160)
np.save('sa_Crho160.npy', sa_Crho160)
np.save('sa_Cbsq160.npy', sa_Cbsq160)

Check if coeffs are working properly: Use Te's long equation to get a list of Te values and calculate varTe

In [ ]:
# test
pg = np.random.rand(CPg1.shape)
rho = np.random.rand(Crho.shape)
bsq = np.random.rand(Cbsq1.shape)

In [ ]:
Te_calc = 

If they are: Do the var(Te) [two versions: 1) calculated from error propagation and 2) directly calculated] VS theta ([0, pi]) plots at different radius (i.e. different index slices of that (288, 128, 128) array). Say, radius r=2, then choose the slice varTe[2, :, 0] so that you got a 1D var(Te) at radius r=2 along theta